# 구글 플레이스토어 리뷰 크롤링
### df_reviews 변수에 크롤링한 데이터 저장
##### Temp 경로에 저장  -> 디렉토리에 크롤링한 파일을 저장했다가 추후 불러오므로 주의

In [1]:
from google_play_scraper import reviews, search
import pandas as pd
from google_play_scraper import Sort

def scrap_reviews(app_name):
    """
    주어진 앱 이름으로부터 리뷰를 수집하고, 스코어와 컨텐트만 남기는 함수입니다.
    매개변수:
    app_name (str): 리뷰를 수집할 앱의 이름입니다.
    반환값:
    df_reviews (DataFrame): 수집된 리뷰의 스코어와 컨텐트를 담은 데이터프레임입니다.
    """
    # 해당 앱의 검색 결과를 가져옵니다.
    search_results = search(app_name, lang="ko", country="kr", n_hits=1)
    app_id = search_results[0]['appId']

    # 앱 ID를 사용하여 한국어 리뷰를 가져옵니다.
    reviews_list = []
    continuation_token = None
    while True: # 모든 리뷰를 가져올 때까지 반복합니다.
        result, continuation_token = reviews(
            app_id,
            count=50, # 한 번에 가져올 리뷰의 수입니다.
            lang='ko',
            country='kr',
            sort=Sort.NEWEST,
            continuation_token=continuation_token
        )
        # 만약 더 이상 리뷰가 없다면 반복문을 종료합니다.
        if not result:
            break
        reviews_list.extend(result)

    # 가져온 리뷰가 없다면 빈 데이터프레임을 반환합니다.
    if len(reviews_list) == 0:
        return pd.DataFrame()

    # 가져온 리뷰를 DataFrame으로 변환합니다.
    df_reviews = pd.DataFrame(reviews_list)

    # 스코어와 컨텐트, 추천수만 남깁니다.
    df_reviews = df_reviews[['score', 'content', 'thumbsUpCount']]

    # 각 앱에 대해 수집된 리뷰 수를 출력합니다.
    print(f"Collected {len(reviews_list)} reviews for '{app_name}'")

    return df_reviews

app_name = input('분석할 앱을 지정해주세요 : ') 
df_reviews = scrap_reviews(app_name)
df_reviews.to_csv('C:/Temp/df_reviews.csv')

분석할 앱을 지정해주세요 : 아키에이지 워
Collected 4353 reviews for '아키에이지 워'


In [2]:
df_reviews['num'] = df_reviews.index

# 'num' 컬럼을 맨 앞으로 옮깁니다.
cols = df_reviews.columns.tolist()
cols = [cols[-1]] + cols[:-1]
df_reviews = df_reviews[cols]

print(df_reviews.head())


   num  score                                            content  \
0    0      4                                          일단해보시길...   
1    1      3                                              재밌어요~   
2    2      3  재미는 있는데...돈 많이 써야되는건 타 게임들 비슷 한데... 뭔놈에 게임이 베터...   
3    3      4                                          좋아요 재미있어요   
4    4      5                                              재미있어요   

   thumbsUpCount  
0              0  
1              0  
2              0  
3              0  
4              0  


### num 컬럼이 생성된 이 데이터 프레임을 다시 저장합니다 

In [3]:
df_reviews.to_csv('C:/Temp/df_reviews_num.csv')

### 키워드 선언¶
#### 키워드를 추가합니다(긍/부정에 나타날 키워드)

In [7]:
# 게임 리뷰에서 자주 사용되는 키워드와 그 변형어를 그룹화한 리스트
# 예를 들면, '렉'은 '랙'이나 '잔렉'이라는 단어로도 사용될 수 있습니다.

keywords_grouped = [
    ['렉', '랙', '잔렉'],
    ['업데이트', '업뎃'],
    ['패치'],
    ['팅', '팅김', '튕김'],
    ['로딩'],
    ['rpg', '알피지'],
    ['섭종'],
    ['스토리'],
    ['캐릭터', '케릭', '캐릭'],
    ['데미지', '뎀', '뎀지', '퍼뎀', '추뎀'],
    ['딜', '딜량'],
    ['콘셉트', '컨셉', '콘셉'],
    ['BGM', 'bgm', '브금', '배경음악'],
    ['노래', '음악'],
    ['의상', '코디', '아바타', '옷', '코스튬', '스킨'],
    ['커스터마이징', '커마'],
    ['자유도'],
    ['컨텐츠', '콘텐츠'],
    ['플레티넘', '플래', '플레'],
    ['챌린저', '챌린져', '챌', '첼린져'],
    ['다이아몬드', '다이아', '다야'],
    ['아처', '아쳐'],
    ['서포터', '서폿'],
    ['갓겜'],
    ['레전드', '래전드'],
    ['가챠', '가차', '갸챠', '갸차'],
    ['무과금'],
    ['과금', '현질'],
    ['일러스트', '일러'],
    ['튜토리얼', '듀토리얼', '튜토'],
    ['유튜브', '유툽', '유튭', '유투브'],
    ['광고'],
    ['그래픽', '글픽', '화질'],
    ['밸런스', '벨붕', '벨런스', '밸붕'],
    ['조작감', '조작', '컨트롤'],
    ['ai', 'AI'],
    ['버그', '오류', '에러'],
    ['데이터'],
    ['와이파이'],
    ['난이도', '하드코어', '어렵', '쉽'],
    ['억까', '어까'],
    ['노잼', '누잼', '개노잼'],
    ['스릴', '박진감'],
    ['짭퉁', '짝퉁', '짭', '카피', '표절'],
    ['꿀잼', '졸잼', '꾸르잼', '개꾸르잼'],
    ['트롤', '비매너', '개트롤'],
    ['리듬', '박자'],
    ['랭크'],
    ['미션'],
    ['중독성'],
    ['루비', '보석', '유료재화', '크리스탈'],
    ['시스템', '구조'],
    ['자동', '오토'],
    ['타격감', '손맛'],
    ['업글', '업그레이드'],
    ['핵', '핵쟁이', '핵유저', '에디터', '중국'],
    ['편의성'],
    ['불친절', '가이드'],
    ['최적화', '발열'],
    ['전술'],
    ['운영'],
    ['소통'],
    ['현실감'],
    ['네트워크', '인터넷'],
    ['킬링타임']
]
# 위에서 그룹화한 키워드들 중 각 그룹의 대표 키워드(첫 번째 키워드)만을 추출하여 저장할 리스트
keywords = []
# 각 그룹의 대표 키워드만 추출하는 반복문
for item in keywords_grouped:
    # item이 리스트인 경우 (즉, 유사어가 있는 경우)
    if isinstance(item, list):
        # 대표 키워드만 keywords 리스트에 추가
        keywords.append(item[0])
         # item이 단일 문자열인 경우 (즉, 유사어가 없는 경우)
    else:
        # 해당 키워드를 그대로 keywords 리스트에 추가
        keywords.append(item)
        

### 세부평점 도출
##### keyword_scores 변수에 각 키워드별 스코어를 딕셔너리 형태로 저장

In [8]:
# 딕셔너리를 사용하여 키워드별 평균 점수 저장
keyword_scores = {}  # 키워드별 평균 점수를 저장할 딕셔너리

# 모든 키워드에 대하여 반복
for keyword in keywords:
    # 리뷰 내용 중 키워드가 포함된 리뷰만 추출
    keyword_reviews = df_reviews[df_reviews['content'].str.contains(keyword)]
    
    # 해당 키워드로 필터링 된 리뷰가 없는 경우, 다음 키워드로 넘어가기
    if keyword_reviews.empty:
        print(f"'{keyword}'에 대한 리뷰를 찾을 수 없습니다.")
        continue

    # 해당 키워드로 필터링 된 리뷰들의 평균 점수 계산
    avg_score = keyword_reviews['score'].mean()
    # 딕셔너리의 key값을 "키워드_score" 형태로 저장하기 위한 문자열 생성
    key_name = f"{keyword}_score"
    # 계산한 평균 점수를 딕셔너리에 저장
    keyword_scores[key_name] = avg_score
    # 결과를 소수점 두 자리까지만 출력
    print(f"{key_name}: {avg_score:.2f}")

# 최종 키워드별 평균 점수 출력
print(keyword_scores)



렉_score: 2.72
업데이트_score: 2.33
패치_score: 1.50
팅_score: 2.34
로딩_score: 2.12
rpg_score: 3.29
'섭종'에 대한 리뷰를 찾을 수 없습니다.
스토리_score: 3.81
캐릭터_score: 3.00
데미지_score: 1.00
딜_score: 2.00
'콘셉트'에 대한 리뷰를 찾을 수 없습니다.
'BGM'에 대한 리뷰를 찾을 수 없습니다.
'노래'에 대한 리뷰를 찾을 수 없습니다.
의상_score: 4.50
커스터마이징_score: 2.75
자유도_score: 5.00
컨텐츠_score: 3.45
'플레티넘'에 대한 리뷰를 찾을 수 없습니다.
'챌린저'에 대한 리뷰를 찾을 수 없습니다.
'다이아몬드'에 대한 리뷰를 찾을 수 없습니다.
'아처'에 대한 리뷰를 찾을 수 없습니다.
'서포터'에 대한 리뷰를 찾을 수 없습니다.
갓겜_score: 4.59
레전드_score: 3.00
가챠_score: 2.00
무과금_score: 3.52
과금_score: 2.99
'일러스트'에 대한 리뷰를 찾을 수 없습니다.
튜토리얼_score: 1.55
유튜브_score: 2.33
광고_score: 1.60
그래픽_score: 3.94
밸런스_score: 1.00
조작감_score: 3.43
ai_score: 1.75
버그_score: 2.67
데이터_score: 2.29
와이파이_score: 2.67
난이도_score: 3.00
'억까'에 대한 리뷰를 찾을 수 없습니다.
노잼_score: 1.29
스릴_score: 4.60
'짭퉁'에 대한 리뷰를 찾을 수 없습니다.
꿀잼_score: 4.97
트롤_score: 3.64
'리듬'에 대한 리뷰를 찾을 수 없습니다.
'랭크'에 대한 리뷰를 찾을 수 없습니다.
미션_score: 3.43
중독성_score: 4.50
'루비'에 대한 리뷰를 찾을 수 없습니다.
시스템_score: 2.92
자동_score: 3.15
타격감_score: 4.46
업글_score: 2.67
핵_sco

## 입력 데이터 전처리
#### 형태소 분석 후 모델 및 벡터라이즈 불러와서 예측 (긍/부정)

In [9]:
# 입력 데이터 전처리

# 필요한 라이브러리 및 모듈을 임포트합니다.
import warnings
from sklearn.neural_network import MLPClassifier, MLPRegressor
from joblib import load
import rhinoMorph
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import load

# 경고 메시지를 무시합니다.
warnings.filterwarnings("ignore")

# CSV 파일을 읽어 데이터프레임으로 변환합니다.
data = pd.read_csv('C:/Temp/df_reviews_num.csv')  # 평점 3,4 제거 안한 data 가져오기

# 불필요한 컬럼 제거
del data['Unnamed: 0'] 

# Rhino 형태소 분석기 로딩
rn = rhinoMorph.startRhino()

# 불용어 사전을 정의합니다. 
# 이 불용어들은 형태소 분석 후 결과에서 제거될 단어들입니다.
stopwords = ["가", "가까스로", "가령", "각", "각각", "각자", "각종", "갖고말하자면", "같다", "같이",
    "개의치않고", "거니와", "거바", "거의", "것", "것과 같이", "것들", "게다가", "게우다", "겨우",
    "견지에서", "결과에 이르다", "결국", "결론을 낼 수 있다", "겸사겸사", "고려하면", "고로", "곧",
    "공동으로", "과", "과연", "관계가 있다", "관계없이", "관련이 있다", "관하여", "관한", "관해서는",
    "구", "구체적으로", "구토하다", "그", "그들", "그때", "그래", "그래도", "그래서", "그러나",
    "그러니", "그러니까", "그러면", "그러므로", "그러한즉", "그런 까닭에", "그런데", "그런즉",
    "그럼", "그럼에도 불구하고", "그렇게 함으로써", "그렇지", "그렇지 않다면", "그렇지 않으면",
    "그렇지만", "그렇지않으면", "그리고", "그리하여", "그만이다", "그에 따르는", "그위에", "그저",
    "그중에서", "그치지 않다", "근거로", "근거하여", "기대여", "기점으로", "기준으로", "기타",
    "까닭으로", "까악", "까지", "까지 미치다", "까지도", "꽈당", "끙끙", "끼익", "나", "나머지는",
    "남들", "남짓", "너", "너희", "너희들", "네", "넷", "년", "논하지 않다", "놀라다", "누가 알겠는가",
    "누구", "다른", "다른 방면으로", "다만", "다섯", "다소", "다수", "다시 말하자면", "다시말하면",
    "다음", "다음에", "다음으로", "단지", "답다", "당신", "당장", "대로 하다", "대하면", "대하여",
    "대해 말하자면", "대해서", "댕그", "더구나", "더군다나", "더라도", "더불어", "더욱더", "더욱이는",
    "도달하다", "도착하다", "동시에", "동안", "된바에야", "된이상", "두번째로", "둘", "둥둥", "뒤따라",
    "뒤이어", "든간에", "들", "등", "등등", "딩동", "따라", "따라서", "따위", "따지지 않다",
    "딱", "때", "때가 되어", "때문에", "또", "또한", "뚝뚝", "라 해도", "령", "로", "로 인하여",
    "로부터", "로써", "륙", "를", "마음대로", "마저", "마저도", "마치", "막론하고", "만 못하다",
    "만약", "만약에", "만은 아니다", "만이 아니다", "만일", "만큼", "말하자면", "말할것도 없고",
    "매", "매번", "메쓰겁다", "몇", "모", "모두", "무렵", "무릎쓰고", "무슨", "무엇", "무엇때문에",
    "물론", "및", "바꾸어말하면", "바꾸어말하자면", "바꾸어서 말하면", "바꾸어서 한다면", "바꿔 말하면",
    "바로", "바와같이", "밖에 안된다", "반대로", "반대로 말하자면", "반드시", "버금", "보는데서",
    "보다더", "보드득", "본대로", "봐", "봐라", "부류의 사람들", "부터", "불구하고", "불문하고",
    "붕붕", "비걱거리다", "비교적", "비길수 없다", "비로소", "비록", "비슷하다", "비추어 보아",
    "비하면", "뿐만 아니라", "뿐만아니라", "뿐이다", "삐걱", "삐걱거리다", "사", "삼", "상대적으로 말하자면",
    "생각한대로", "설령", "설마", "설사", "셋", "소생", "소인", "솨", "쉿", "습니까", "습니다",
    "시각", "시간", "시작하여", "시초에", "시키다", "실로", "심지어", "아", "아니", "아니나다를가",
    "아니라면", "아니면", "아니었다면", "아래윗", "아무거나", "아무도", "아야", "아울러", "아이",
    "아이고", "아이구", "아이야", "아이쿠", "아하", "아홉", "안 그러면", "않기 위하여", "않기 위해서",
    "알 수 있다", "알았어", "앗", "앞에서", "앞의것", "야", "약간", "양자", "어", "어기여차",
    "어느", "어느 년도", "어느것", "어느곳", "어느때", "어느쪽", "어느해", "어디", "어때", "어떠한",
    "어떤", "어떤것", "어떤것들", "어떻게", "어떻해", "어이", "어째서", "어쨋든", "어쩔수 없다",
    "어찌", "어찌됏든", "어찌됏어", "어찌하든지", "어찌하여", "언제", "언젠가", "얼마", "얼마 안 되는 것",
    "얼마간", "얼마나", "얼마든지", "얼마만큼", "얼마큼", "엉엉", "에", "에 가서", "에 달려 있다",
    "에 대해", "에 있다", "에 한하다", "에게", "에서", "여", "여기", "여덟", "여러분", "여보시오",
    "여부", "여섯", "여전히", "여차", "연관되다", "연이서", "영", "영차", "옆사람", "예", "예를 들면",
    "예를 들자면", "예컨대", "예하면", "오", "오로지", "오르다", "오자마자", "오직", "오호", "오히려",
    "와", "와 같은 사람들", "와르르", "와아", "왜", "왜냐하면", "외에도", "요만큼", "요만한 것",
    "요만한걸", "요컨대", "우르르", "우리", "우리들", "우선", "우에 종합한것과같이", "운운", "월",
    "위에서 서술한바와같이", "위하여", "위해서", "윙윙", "육", "으로", "으로 인하여", "으로서", "으로써",
    "을", "응", "응당", "의", "의거하여", "의지하여", "의해", "의해되다", "의해서", "이", "이 되다",
    "이 때문에", "이 밖에", "이 외에", "이 정도의", "이것", "이곳", "이때", "이라면", "이래", "이러이러하다",
    "이러한", "이런", "이럴정도로", "이렇게 많은 것", "이렇게되면", "이렇게말하자면", "이렇구나", "이로 인하여",
    "이르기까지", "이리하여", "이만큼", "이번", "이봐", "이상", "이어서", "이었다", "이와 같다",
    "이와 같은", "이와 반대로", "이와같다면", "이외에도", "이용하여", "이유만으로", "이젠", "이지만",
    "이쪽", "이천구", "이천육", "이천칠", "이천팔", "인 듯하다", "인젠", "일", "일것이다", "일곱",
    "일단", "일때", "일반적으로", "일지라도", "임에 틀림없다", "입각하여", "입장에서", "잇따라", "있다",
    "자", "자기", "자기집", "자마자", "자신", "잠깐", "잠시", "저", "저것", "저것만큼", "저기", "저쪽",
    "저희", "전부", "전자", "전후", "점에서 보아", "정도에 이르다", "제", "제각기", "제외하고", "조금",
    "조차", "조차도", "졸졸", "좀", "좋아", "좍좍", "주룩주룩", "주저하지 않고", "줄은 몰랏다", "줄은모른다",
    "중에서", "중의하나", "즈음하여", "즉", "즉시", "지각", "지금", "지든지", "지만", "지말고",
    "진짜로", "쪽으로", "차라리", "참", "참나", "첫번째로", "쳇", "총적으로", "총적으로 말하면",
    "총적으로 보면", "칠", "콸콸", "쾅쾅", "쿵", "타다", "타인", "탕탕", "토하다", "통하여",
    "툭", "퉤", "틈타", "팍", "팔", "퍽", "펄렁", "하", "하게될것이다", "하게하다", "하겠는가",
    "하고 있다", "하고있었다", "하곤하였다", "하구나", "하기 때문에", "하기 위하여", "하기는한데",
    "하기만 하면", "하기보다는", "하기에", "하나", "하느니", "하는 김에", "하는 편이 낫다",
    "하는것도", "하는것만 못하다", "하는것이 낫다", "하는바", "하더라도", "하도다", "하도록시키다",
    "하도록하다", "하든지", "하려고하다", "하마터면", "하면 할수록", "하면된다", "하면서", "하물며",
    "하여금", "하여야", "하자마자", "하지 않는다면", "하지 않도록", "하지마", "하지마라", "하지만",
    "하하", "한 까닭에", "한 이유는", "한 후", "한다면", "한다면 몰라도", "한켠으로는", "한항목",
    "할 따름이다", "할 생각이다", "할 줄 안다", "할 지경이다", "할 힘이 있다", "할때", "할만하다",
    "할망정", "할뿐", "할수있다", "할수있어", "할줄알다", "할지라도", "할지언정", "함께", "해도된다",
    "해도좋다", "해봐요", "해서는 안된다", "해야한다", "해요", "했어요", "향하다", "향하여", "향해서",
    "허", "허걱", "허허", "헉", "헉헉", "헐떡헐떡", "형식으로 쓰여", "혹시", "혹은", "혼자",
    "훨씬", "휘익", "휴", "흐흐", "흥", "힘입어",'ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ',
    'ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ'
]

def analyze_pos(df, stopwords=stopwords):   # 형태소 분석 함수
    new_data = []

    # 데이터프레임의 각 행에 대해서 반복합니다.
    for i, row in df.iterrows():
        try:
            data_each = str(row['content'])
            data_score = row['score']
            data_tuc = row['thumbsUpCount']
            data_num = row['num']  # num 컬럼 값을 가져옵니다.

            # 형태소 분석 및 품사 태깅을 진행합니다.
            morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each, pos=['NNG', 'VV', 'VA', 'XR', 'IC', 'MAG','EC','MAJ'], eomi=True, xrVv=True)

            # 결과에서 불용어를 제거합니다.
            if stopwords:
                morphed_data_each = [word for word in morphed_data_each if word not in stopwords]

            # 형태소들을 공백을 기준으로 합쳐서 하나의 문자열로 만듭니다.
            merged_data_each = ' '.join(morphed_data_each)

            # 형태소 분석 결과가 비어있지 않으면 새로운 데이터 목록에 추가합니다.
            if merged_data_each:
                new_data.append([data_score, merged_data_each, data_tuc, data_num])  # num 컬럼 값도 추가합니다.
        except Exception as e:
            # 오류 발생 시 오류 메시지를 출력합니다.
            print(f"Error occurred at index {i}: {e}")
            continue

    # 새로운 데이터 목록을 데이터프레임으로 변환하여 반환합니다.
    df_new = pd.DataFrame(new_data, columns=['score', 'content', 'thumbsUpCount', 'num'])  # num 컬럼도 포함

    return df_new

# 형태소 분석 수행
data_analyzed = analyze_pos(data, stopwords)

# TF-IDF 벡터라이저 로드
# 사전에 학습된 TF-IDF 벡터라이저를 joblib로부터 불러옵니다.
tfidf_vectorizer = load("C:/Users/AI/Desktop/Hagsim_project/new_project/csv/Project R/Project_R/tfidf_vectorizer.joblib")

# 형태소 분석이 완료된 'content' 칼럼을 이용하여 TF-IDF 변환을 수행합니다.
# 이 변환을 통해 리뷰 텍스트를 수치 데이터로 바꾸어줍니다.
tfidf_matrix = tfidf_vectorizer.transform(data_analyzed['content'])

# 변환된 TF-IDF 행렬을 데이터프레임 형식으로 변환합니다.
# 이렇게 하면 각 단어별 TF-IDF 점수를 볼 수 있습니다.
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names())

# 미리 학습된 로지스틱 회귀 모델을 joblib로부터 불러옵니다.
model = load("C:/Users/AI/Desktop/Hagsim_project/new_project/csv/Project R/Project_R/logreg2.joblib")

# TF-IDF로 변환된 리뷰 텍스트 데이터를 사용하여 긍정/부정 예측을 수행합니다.
predictions = model.predict(tfidf_df)

# 예측 결과를 'sentiment'라는 새로운 칼럼에 저장합니다.
# 여기서 'sentiment' 칼럼에는 각 리뷰에 대한 긍정/부정 예측 결과가 저장됩니다.
data_analyzed['sentiment'] = predictions


C:\Users\AI\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


filepath:  C:\Users\AI\Anaconda3\lib\site-packages
classpath:  C:\Users\AI\Anaconda3\lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!


### 데이터 분할 및 전체 리뷰에서 빈도가 높은 키워드 5개 출력

In [11]:
# 긍정 리뷰와 부정 리뷰를 나누어 분석할 데이터프레임 생성
# 'sentiment' 칼럼의 값이 1인 경우 긍정 리뷰, -1인 경우 부정 리뷰로 판단하여 데이터를 분리합니다.
data_positive = data_analyzed[data_analyzed['sentiment'] == 1]
data_negative = data_analyzed[data_analyzed['sentiment'] == -1]

# 긍정 리뷰에서 빈도가 높은 키워드 5개 추출
# 각 키워드 그룹의 첫 번째 키워드를 딕셔너리의 키로 사용하고 빈도수를 0으로 초기화합니다.
keyword_frequencies_pos = {keyword_group[0]: 0 for keyword_group in keywords_grouped}
for content in data_positive['content']:
    for keyword_group in keywords_grouped:
        # 각 리뷰 내용에서 해당 키워드 그룹의 키워드가 몇 번 등장하는지 카운트합니다.
        group_frequency = sum(keyword in content for keyword in keyword_group)
        keyword_frequencies_pos[keyword_group[0]] += group_frequency
# 빈도수를 기준으로 상위 5개의 키워드를 선택합니다.
top_5_keywords_pos = dict(sorted(keyword_frequencies_pos.items(), key=lambda x: x[1], reverse=True)[:5])
print("상위 5개의 긍정 리뷰 키워드:", top_5_keywords_pos)

# 부정 리뷰에서 빈도가 높은 키워드 5개 추출
# 부정 리뷰 중 'content' 칼럼에 NaN값이 있는 행을 제거합니다.
data_negative.dropna(subset=['content'], inplace=True)
# 각 키워드 그룹의 첫 번째 키워드를 딕셔너리의 키로 사용하고 빈도수를 0으로 초기화합니다.
keyword_frequencies_neg = {keyword_group[0]: 0 for keyword_group in keywords_grouped}
for content in data_negative['content']:
    for keyword_group in keywords_grouped:
        # 각 리뷰 내용에서 해당 키워드 그룹의 키워드가 몇 번 등장하는지 카운트합니다.
        group_frequency = sum(keyword in content for keyword in keyword_group)
        keyword_frequencies_neg[keyword_group[0]] += group_frequency
# 빈도수를 기준으로 상위 5개의 키워드를 선택합니다.
top_5_keywords_neg = dict(sorted(keyword_frequencies_neg.items(), key=lambda x: x[1], reverse=True)[:5])
print("상위 5개의 부정 리뷰 키워드:", top_5_keywords_neg)



상위 5개의 긍정 리뷰 키워드: {'그래픽': 169, '타격감': 80, '과금': 59, '캐릭터': 58, '난이도': 50}
상위 5개의 부정 리뷰 키워드: {'최적화': 198, '렉': 162, '과금': 128, '캐릭터': 90, '그래픽': 72}


### 키워드 별로 추천 기준 상위 10개의 댓글을 가져옴

In [12]:
# 키워드 그룹을 찾는 함수
def get_keyword_group(keyword):
    # 주어진 키워드가 어느 키워드 그룹에 속하는지를 확인합니다.
    for group in keywords_grouped:
        if keyword in group:
            return group
    # 해당 키워드가 특정 그룹에 속하지 않을 경우 그 키워드만 반환합니다.
    return [keyword]

# 긍정 리뷰에 대한 상위 10개 댓글 가져오기
top_10_reviews_by_keyword_pos = []
# 상위 5개 긍정 키워드를 순회합니다.
for key_pos, frequency in top_5_keywords_pos.items():
    # 해당 키워드가 속한 그룹을 찾습니다.
    key_pos_group = get_keyword_group(key_pos)
    # 그룹 내의 키워드들을 정규식 패턴으로 변환합니다.
    keyword_pattern = "|".join(key_pos_group)
    # 해당 키워드 패턴이 포함된 긍정 리뷰들을 필터링합니다.
    keyword_reviews = data_positive[data_positive['content'].str.contains(keyword_pattern, regex=True)]
    # 'thumbsUpCount' 기준으로 리뷰들을 내림차순으로 정렬합니다.
    sorted_reviews = keyword_reviews.sort_values(by='thumbsUpCount', ascending=False)
    # 상위 10개의 리뷰 내용만을 추출합니다.
    top_10_reviews = sorted_reviews.head(10)['content'].tolist()
    # 결과를 리스트에 추가합니다.
    top_10_reviews_by_keyword_pos.append(top_10_reviews)

# 부정 리뷰에 대한 상위 10개 댓글 가져오기
top_10_reviews_by_keyword_neg = []
# 상위 5개 부정 키워드를 순회합니다.
for key_neg, frequency in top_5_keywords_neg.items():
    # 해당 키워드가 속한 그룹을 찾습니다.
    key_neg_group = get_keyword_group(key_neg)
    # 그룹 내의 키워드들을 정규식 패턴으로 변환합니다.
    keyword_pattern = "|".join(key_neg_group)
    # 해당 키워드 패턴이 포함된 부정 리뷰들을 필터링합니다.
    keyword_reviews = data_negative[data_negative['content'].str.contains(keyword_pattern, regex=True)]
    # 'thumbsUpCount' 기준으로 리뷰들을 내림차순으로 정렬합니다.
    sorted_reviews = keyword_reviews.sort_values(by='thumbsUpCount', ascending=False)
    # 상위 10개의 리뷰 내용만을 추출합니다.
    top_10_reviews = sorted_reviews.head(10)['content'].tolist()
    # 결과를 리스트에 추가합니다.
    top_10_reviews_by_keyword_neg.append(top_10_reviews)


### 그 형태소 분석된 데이터와 원본 데이터 리뷰 비교

In [13]:
# 선택한 리뷰를 원본 데이터에서 찾아서 보여주는 함수
def show_original_review(selected_review_num, original_data):
    # 원본 데이터에서 선택한 리뷰 번호에 해당하는 리뷰 내용을 찾습니다.
    selected_review = original_data[original_data['num'] == selected_review_num]['content'].values[0]
    # 해당 리뷰 내용을 출력합니다.
    print("리뷰 내용입니다:")
    print(selected_review)
    print()

# 각 키워드별로 상위 10개 리뷰를 출력하면서 선택한 리뷰의 원본을 보여주는 부분
# 긍정 키워드에 대한 상위 10개 리뷰 출력
for i, (key_pos, reviews) in enumerate(zip(top_5_keywords_pos.keys(), top_10_reviews_by_keyword_pos)):
    print(f"Keyword {i+1}: {key_pos}")
    for j, review in enumerate(reviews, start=1):
        # 리뷰 내용을 출력합니다.
        print(f"Review {j}: {review}")
        # 해당 리뷰의 번호(num) 값을 찾습니다.
        selected_review_num = data_positive[data_positive['content'] == review]['num'].values[0]
        # 원본 데이터에서 해당 리뷰의 내용을 보여주는 함수를 호출합니다.
        show_original_review(selected_review_num, data)
    print()

# 부정 키워드에 대한 상위 10개 리뷰 출력
for i, (key_neg, reviews) in enumerate(zip(top_5_keywords_neg.keys(), top_10_reviews_by_keyword_neg)):
    print(f"Keyword {i+1}: {key_neg}")
    for j, review in enumerate(reviews, start=1):
        # 리뷰 내용을 출력합니다.
        print(f"Review {j}: {review}")
        # 해당 리뷰의 번호(num) 값을 찾습니다.
        selected_review_num = data_negative[data_negative['content'] == review]['num'].values[0]
        # 원본 데이터에서 해당 리뷰의 내용을 보여주는 함수를 호출합니다.
        show_original_review(selected_review_num, data)
    print()


Keyword 1: 그래픽
Review 1: 모바일 게임 크다 기대 하다 지 는데 상당히 완성도 게 나오다 아서 그래픽 그래픽 세세하다 커스터마이징 잘 되다 최적화 깔끔하다 정말 좋다 특히 모바일 깔끔하다 조작 감 정말 ㅋㅋ 앞 운영 잘 하다 ㄴ다면 초 갓겜 되다 지
리뷰 내용입니다:
모바일 게임으로서는 큰 기대를 하지 않았는데 상당히 완성도 있게 나와서 놀랐습니다. 그래픽도 그래픽이지만 세세한 커스터마이징, 나름 잘된 최적화와 깔끔한 UI도 정말 좋네요. 특히 모바일임에도 깔끔한 조작감에 정말 놀랐습니다ㅋㅋ 앞으로 운영만 잘한다면 초갓겜이 될 수 있지 않을까 싶네요

Review 2: 재미있다 그래픽 좋다 고 캐릭터 이쁘다 고
리뷰 내용입니다:
재미있을거 같아요 그래픽 좋고 캐릭터도 이쁘고

Review 3: 가다 주다 모르다 게임 그래픽 며 캐릭터 아이템 지 시스템 잘 구축 되다 고 슈트 갈아입다 으면서 다양 킬 접다 하다 최고 장점 별 고 ^^ 정말 감사 하다 게임 만들다 어서 깊다 노고 진심 느끼다 앞 더 번창하다 ^^
리뷰 내용입니다:
시간 가는줄 모르는 게임입니다~!~그래픽이며 캐릭 아이템 여러가지 시스템으로 잘 구축이되어있고 슈트를3개씩이나 갈아입으면서 다양한스킬들을 접할수 있어 최고의 장점입니다!! 별100개드리고싶습니다!!^^ 정말감사합니다 이런게임을 만들어주셔서 깊은노고가 진심으로 느껴집니다 앞으로도 더 번창하세요~!~!^^

Review 4: 그래픽 잘 돌아가다 신기하다 이쁘다 캐릭터 화면 구성 레이아웃 완벽 중독성 재미 글쎄 아직 오랫만 오래 해보다 고 게임 나이스
리뷰 내용입니다:
쩌는 그래픽 잘돌아가는게 신기하다. 이쁜 캐릭터와 화면 구성 레이아웃이 완벽 ㅎ 중독성 있는 재미는 글쎄, 아직 ㅎ 하지만 오랫만에 오래 해보고 싶은 게임 나이스

Review 5: 그래픽 너무 좋다 최적화 나쁘다 레 검다 사막 모바일 합치다 놓다 느낌 시작 하다 ㄴ지 과금 잘 모르다 PC 하다 모바일 하다 아도 재미있다
리뷰 내용입니다:
그래픽 너무 좋음. 

### 원본 리뷰만 출력되게 하기(검색) 
#### 긍정 빈도가 높은 키워드면 긍정 리뷰만
#### 부정 빈도가 높은 키워드면 부정 리뷰만
#### 긍부정 모두 포함되는 키워드면 함께 확인할 수 있게 긍부정 리뷰가 한번에 나타납니다

In [15]:
# 원본 리뷰를 출력하는 함수
def show_original_review(selected_review_num, original_data):
    """ 
    원본 데이터에서 선택한 리뷰 번호에 해당하는 리뷰 내용을 찾아 출력하는 함수
    
    :param selected_review_num: 선택한 리뷰의 번호
    :param original_data: 원본 데이터
    """
    selected_review = original_data[original_data['num'] == selected_review_num]['content'].values[0]
    print("리뷰 내용입니다:")
    print(selected_review)
    print()

def get_keyword_group(keyword, keyword_groups):
    """ 
    주어진 키워드에 해당하는 키워드 그룹을 반환하는 함수
    
    :param keyword: 검색할 키워드
    :param keyword_groups: 키워드 그룹 정보
    :return: 키워드에 해당하는 키워드 그룹
    """
    for group in keyword_groups:
        if keyword in group:
            return group
    return [keyword]

def get_top_reviews_by_keyword(keyword, data_source, keyword_groups):
    """ 
    주어진 키워드를 포함하는 리뷰 중에서 상위 10개의 리뷰를 반환하는 함수
    
    :param keyword: 검색할 키워드
    :param data_source: 검색할 데이터 (긍정 또는 부정 리뷰 데이터)
    :param keyword_groups: 키워드 그룹 정보
    :return: 상위 10개의 리뷰 내용 리스트
    """
    keyword_group = get_keyword_group(keyword, keyword_groups)  
    keyword_pattern = "|".join(keyword_group)
    keyword_reviews = data_source[data_source['content'].str.contains(keyword_pattern, regex=True)]
    sorted_reviews = keyword_reviews.sort_values(by='thumbsUpCount', ascending=False)
    return sorted_reviews.head(10)['content'].tolist()

def show_reviews_by_keyword(selected_keyword, data_positive, data_negative, keyword_groups):
    """ 
    선택한 키워드를 포함하는 긍정/부정 리뷰를 출력하는 함수
    
    :param selected_keyword: 검색어
    :param data_positive: 긍정 리뷰 데이터
    :param data_negative: 부정 리뷰 데이터
    :param keyword_groups: 키워드 그룹 정보
    """
    pos_reviews = get_top_reviews_by_keyword(selected_keyword, data_positive, keyword_groups)
    neg_reviews = get_top_reviews_by_keyword(selected_keyword, data_negative, keyword_groups)

    if not pos_reviews and not neg_reviews:
        print("해당 키워드를 포함하는 리뷰가 없습니다.")
        return

    if selected_keyword in top_5_keywords_neg.keys() and selected_keyword not in top_5_keywords_pos.keys():
        pos_reviews = []
    elif selected_keyword in top_5_keywords_pos.keys() and selected_keyword not in top_5_keywords_neg.keys():
        neg_reviews = []

    if pos_reviews:
        print("긍정 리뷰:")
        for review in pos_reviews:
            review_num = data_positive[data_positive['content'] == review]['num'].values[0]
            show_original_review(review_num, data)

    if neg_reviews:
        print("\n부정 리뷰:")
        for review in neg_reviews:
            review_num = data_negative[data_negative['content'] == review]['num'].values[0]
            show_original_review(review_num, data)

# 사용자로부터 검색할 키워드를 입력받는 부분
selected_keyword = input("검색어를 입력하세요: ").strip()
show_reviews_by_keyword(selected_keyword, data_positive, data_negative, keywords_grouped)


검색어를 입력하세요: 타격감
긍정 리뷰:
리뷰 내용입니다:
뭔가 컷씬이나 qte같은 것들이 너무 뜬금없고 부자연스럽습니다. 총이나 칼을 사용하는데 타격감이 부드럽지도, 임팩트 있지도 않고 너무 스르륵 지나갑니다. 첫인상이 전혀 강하지않고 인디게임보다도 더 부족하다고 생각됩니다. 카카오게임즈는 개인적으로 응원하는 곳이라 좀 좋은 게임을 만들기를 기대합니다! 업데이트를 기대하겠습니다

리뷰 내용입니다:
다른 게임에 비해선 그래픽 완전 좋고, 타격감 좋고 다 좋네여~ 그런데..!! 단점은 딱 두가지있네요 발열이 심함 배터리 너무 빨리땀 이점만 고쳐주세요

리뷰 내용입니다:
게임이 너무 재미있어요. 타격감도 괜찮고 자사 돌리면서 창 기능을 사용해 드라마나 영화 웹툰 유튜브등 볼수있게 만들어주시고 내가 원하는 모바일게임이 이런느낌이라 좋습니다.

리뷰 내용입니다:
새롭고 넘넘 재밌네여 타격감도 있고 그래픽 끝내주고 다른 의상으로 교체하면 다른무기로도 할수 있고 나오는 인물들로 전환되어 진행도 되고 넘 재미나요~^^

리뷰 내용입니다:
근래 해본것중 타격감이 좋네요. 아직 초반이라그런지 게임 이해도가 낮아서그런지 좀 어려운듯한 느낌이 있음. 의상이 다양하고 예쁨.

리뷰 내용입니다:
재가 이 게임을 잠깐 플레이 했는데 그래픽도 최고 타격감 최고네요 스토리도 탄탄하고요

리뷰 내용입니다:
일단 시작한지는 얼마안됐지만 타격감이나 공속 보조무기등 엑션감이 좋네요...좀더 해봐야 하겠지만 아직은 만족하고 즐기고 있습니다...

리뷰 내용입니다:
그래픽 타격감 손쉬운 사냥 만족

리뷰 내용입니다:
기대했던것 만큼 잘만들어진것 같네요, 그래픽, 커마 는 요즘 출시 겜은 상향평준화 되어 언급할필요 없고, sf장르, 전투 액션 타격감,후판정 전투로 컨트롤 하는재미등 앞으로 잘운영해서 갓게임 되었으면 좋겠습니다.화이팅

리뷰 내용입니다:
그래픽 좋고, 타격감 좋고 근데 업그레이드나 기타시스템이 너무 복잡하고 어렵네요

